In [1]:
import seamsh
from seamsh.geometry import CurveType
import seamsh.geometry
import numpy as np
from osgeo import osr

Error   : Gmsh has not been initialized


In [3]:
domain_srs = osr.SpatialReference()
domain_srs.ImportFromProj4("+proj=utm +ellps=WGS84 +zone=31")
domain = seamsh.geometry.Domain(domain_srs)

In [4]:
domain.add_boundary_curves_shp("data2/data_no_duplicate.shp",
                               "physical", CurveType.POLYLINE)

 * Import features from "data2/data_no_duplicate.shp" * 
98 features imported


In [5]:
domain.add_interior_curves_shp("data2/interior.shp",
                               None, CurveType.STRICTPOLYLINE)
domain.add_interior_points_shp("data2/interior_points.shp", "physical")

 * Import features from "data2/interior.shp" * 
8 features imported
 * Import features from "data2/interior_points.shp" * 
3 features imported


In [6]:
bath_field = seamsh.field.Raster("data2/medit.tiff")

 * Create field from raster file "data2/medit.tiff" * 


In [7]:
dist_coast = seamsh.field.Distance(domain, 100, ["coast", "island"])


 * Create distance field * 
Sampling features for distance computation
98 features sampled
Build KDTree with 6824 points


In [8]:
dist_porquerolles = seamsh.field.Distance(domain, 20, ["porquerolles"])


 * Create distance field * 
Sampling features for distance computation
43 features sampled
Build KDTree with 2676 points


In [9]:
def mesh_size(x, projection):
    s_coast = np.clip((dist_coast(x, projection)-400)*0.5, 200, 5000)
    s_porq = np.clip((dist_porquerolles(x, projection)-200)*0.5, 50, 5000)
    s_dist = np.minimum(s_coast, s_porq)
    return s_dist

In [10]:
output_srs = osr.SpatialReference()
output_srs.ImportFromProj4("+proj=latlon +ellps=WGS84")

seamsh.gmsh.mesh(domain, "gis_mesh.msh", mesh_size,
                 intermediate_file_name="debug", output_srs=output_srs)

 * Generate mesh * 
Build topology
Build gmsh model
Build mesh size field


Mesh with gmsh
Write "gis_mesh.msh" (msh version 4.0)
